In [254]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import shutil
import win32com.client
import time

In [255]:
#Date of report refresh
TodaysDate = time.strftime("%#m/%d/%Y")

In [256]:
#Set path and excel writer for the safety data workbook
xlsx_path = 'C:/Users/vorbej1/desktop/Safety Data.xlsx'
book = load_workbook(xlsx_path)

In [257]:
#Remove sheets from previous report to overwrite with most updated
esd_sheet=book['ESD']
osha_sheet=book['OSHA and NMGC']
book.remove(esd_sheet)
book.remove(osha_sheet)

In [258]:
#Initialize the excel writer
writer = pd.ExcelWriter(xlsx_path, engine = 'openpyxl')
writer.book = book

In [259]:
#Onedrive path for final report
od_path = 'C:/Users/vorbej1/National Grid/Kearns, Joseph M. - Metrics Heirachy/VP Scorecards/'

In [260]:
#Setup application to open excel document
app = win32com.client.Dispatch("Excel.Application")

In [261]:
#Identify sheet in excel workbook
pop_summary = pd.ExcelFile('X:/Data/Population Summary/Pd03 HR Population Summary 6.30.2019.xlsx')
sheet=[item for item in pop_summary.sheet_names if 'Population' in item]

In [262]:
#Read in hierarchy file
hier2 = pd.read_excel('X:/Data/Population Summary/Pd03 HR Population Summary 6.30.2019.xlsx', sheet_name=sheet[0],\
                                                       dtype={'CA Personnel Number':str,'CA Supervisor Personnel Number':str,'Labor type':str,\
                                                              'CA L03 Cost Center':str})

In [263]:
#Read in scraped safety metrics
nmgc = pd.read_excel('X:/Ying_Ba/MH - Safety Metrics/Input/incidents_report.xls')
prev_rtc_osha = pd.read_excel('X:/Ying_Ba/MH - Safety Metrics/Input/ims_report.xls')
esd = pd.read_excel('X:/Ying_Ba/MH - Safety Metrics/Input/asses_perf_report.xls')

In [264]:
#Format NMGC report
nmgc_2 = nmgc.rename(columns={'Employee EMPLID': 'Employee ID', 'Incident Type': 'Incident Class'})
nmgc_2 = nmgc_2[['Employee ID','Incident Class']]

In [265]:
#Format OSHA report
prev_rtc_osha2 = prev_rtc_osha.rename(columns={'Emp ID': 'Employee ID'})
prev_rtc_osha2 = prev_rtc_osha2[['Employee ID', 'Incident Class']]
prev_rtc_osha2['Employee ID'] = prev_rtc_osha2['Employee ID'].astype(str)

In [266]:
#Format ESD report
esd_2 = esd.rename(columns={'EMPLID': 'Employee ID'})
esd_2['Employee ID'] = esd_2['Employee ID'].astype(str)

In [267]:
#Combine osha reportables and nmgc report
osha_and_nmgc = pd.concat([nmgc_2,prev_rtc_osha2])

In [268]:
#Merge hierarchy with combined OSHA/NMGC report
osha_and_nmgc = pd.merge(osha_and_nmgc, hier2, left_on='Employee ID', right_on='CA Personnel Number')

In [269]:
#Initialize empty columns for safety targets & rundate of report
osha_and_nmgc['LTI FYTD Target'] = ''
osha_and_nmgc['Prev RTC FYTD Target'] = ''
osha_and_nmgc['OSHA FYTD Target'] = ''
osha_and_nmgc['Refresh Date'] = TodaysDate
esd_2['Refresh Date'] = TodaysDate

In [270]:
#Apply target numbers - hardcode as they are set for the year
osha_and_nmgc.loc[osha_and_nmgc['CA L04 Cost Center'] == 'Gas Field Operations-NY', 'LTI FYTD Target'] = 7
osha_and_nmgc.loc[osha_and_nmgc['CA L04 Cost Center'] == 'Gas Field Operations-NY', 'Prev RTC FYTD Target'] = 10
osha_and_nmgc.loc[osha_and_nmgc['CA L04 Cost Center'] == 'Gas Field Operations-NY', 'OSHA FYTD Target'] = 27

In [271]:
osha_and_nmgc.loc[osha_and_nmgc['CA L04 Cost Center'] == 'Gas Field Operations-NE', 'LTI FYTD Target'] = 4
osha_and_nmgc.loc[osha_and_nmgc['CA L04 Cost Center'] == 'Gas Field Operations-NE', 'Prev RTC FYTD Target'] = 13
osha_and_nmgc.loc[osha_and_nmgc['CA L04 Cost Center'] == 'Gas Field Operations-NE', 'OSHA FYTD Target'] = 22

In [272]:
osha_and_nmgc.loc[osha_and_nmgc['CA L04 Cost Center'] == 'US Gas Engineering', 'LTI FYTD Target'] = 0
osha_and_nmgc.loc[osha_and_nmgc['CA L04 Cost Center'] == 'US Gas Engineering', 'Prev RTC FYTD Target'] = 3
osha_and_nmgc.loc[osha_and_nmgc['CA L04 Cost Center'] == 'US Gas Engineering', 'OSHA FYTD Target'] = 2

In [273]:
#Write to Safety Metrics workbook
osha_and_nmgc.to_csv('C:/Users/vorbej1/desktop/OSHA and NMGC.csv')
osha_and_nmgc.to_excel(writer, sheet_name = 'OSHA and NMGC')

In [274]:
#Merge hierarchy with ESD report
esd_final = pd.merge(esd_2, hier2, left_on='Employee ID', right_on='CA Personnel Number')

In [275]:
#Write to Safety Metrics workbook
esd_final.to_csv('C:/Users/vorbej1/desktop/ESD and Hier.csv')
esd_final.to_excel(writer, sheet_name = 'ESD')

In [276]:
#Save and close workbook
writer.save()
writer.close()

In [278]:
#Move file to onedrive to be utilized for tableau dashboard
shutil.copyfile(xlsx_path, od_path + xlsx_path[25:])

'C:/Users/vorbej1/National Grid/Kearns, Joseph M. - Metrics Heirachy/VP Scorecards/Safety Data.xlsx'